In [ ]:
 import langgraph

In [ ]:
'''
first i have to make a node edge and then state 
now =
'''

'\nfirst i have to make a node edge and then state \nnow =\n'

In [ ]:
from typing import List, Dict,TypedDict
from langgraph.graph import StateGraph,START,END

class AgentState(TypedDict):
    message:str
    
def greeting_node(state:AgentState)->AgentState:
    ''' This node is used to greet the user '''
    
    state["message"]="Hello, how can I help you today?" + state["message"]
    
    return state

In [ ]:
print(state["message"])

NameError: name 'state' is not defined

In [ ]:
graph=StateGraph(AgentState)
graph.add_node("greeting",greeting_node)
graph.set_entry_point("greeting")
graph.set_finish_point("greeting")
app= graph.compile()



In [ ]:
app.invoke({"message":"baby"})

{'message': 'Hello, how can I help you today?baby'}

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
from typing import Dict,List,Optional

# ---- 1. Define the State ----
# State is a dict-like object that flows between nodes
class AgentState(TypedDict):
    research_mode: bool
    query: str
    documents: Optional[List]
    summary: Optional[str]
    report: Optional[str]
    answer: Optional[str]

# ---- 2. Define Nodes ----

# Direct Answer Node
def direct_answer(state: AgentState):
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    response = llm.invoke(f"Answer this query directly: {state['query']}")
    state["answer"] = response.content
    state["research_mode"] = False  # reset
    return state

# Retriever Node (mock)
def retriever(state: AgentState):
    # Imagine this calls a case law API
    import requests

    def fetch_kanoon_data(user_input, api_token, pagenum=0):
        url = "https://api.indiankanoon.org/search/"
        headers = {"Authorization": f"Token {api_token}", "Accept": "application/json"}
        params = {"formInput": user_input, "pagenum": pagenum}
        try:
            response = requests.post(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            
            # Extract relevant fields and generate case links
            results = []
            for doc in data.get('docs', []):
                case_info = {
                    'tid': doc.get('tid'),
                    'title': doc.get('title'),
                    'docsource': doc.get('docsource'),
                    'publishdate': doc.get('publishdate'),
                    'author': doc.get('author'),
                    # 'bench': doc.get('bench'),
                    # 'catids': doc.get('catids'),
                    'citation': doc.get('citation'),
                    'numcites': doc.get('numcites'),
                    'numcitedby': doc.get('numcitedby'),
                    'link': f"https://indiankanoon.org/doc/{doc.get('tid')}/"
                }
                results.append(case_info)
            
            return {
                'cases': results,
                'found': data.get('found'),
                'encodedformInput': data.get('encodedformInput')
            }
        except requests.exceptions.RequestException as e:
            raise Exception(f"API request failed: {e}")
        
    api_token = "14643c2a5b137ab178234409a2e8b8e219629d60"
    result = fetch_kanoon_data(state['query'], api_token)

    # docs = [f"Case law related to: {state['query']}"]
    state["documents"] = result
    return state

# Summarizer Node
def summarizer(state: AgentState):
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    notes = llm.invoke(
        f"Summarize the following legal documents:\n{state['documents']}"
    )
    state["summary"] = notes.content
    return state

# Formatter Node
def formatter(state: AgentState):
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    schema = """
    Format into sections:
    - Issue
    - Relevant Case Law
    - Analysis
    - Conclusion
    - Citations
    """
    report = llm.invoke(f"{schema}\n\nSummary:\n{state['summary']}")
    state["report"] = report.content
    state["research_mode"] = False  # reset
    return state

# ---- 3. Build the Graph ----
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("direct_answer", direct_answer)
workflow.add_node("retriever", retriever)
workflow.add_node("summarizer", summarizer)
workflow.add_node("formatter", formatter)

# Conditional branch
def decide_branch(state: AgentState):
    if state.get("research_mode"):
        return "retriever"
    else:
        return "direct_answer"

workflow.set_entry_point("deciding node")
workflow.add_conditional_edges("direct_answer", decide_branch, {
    "retriever": "retriever",
    "direct_answer": END
})

# Research path edges
workflow.add_edge("retriever", "summarizer")
workflow.add_edge("summarizer", "formatter")
workflow.add_edge("formatter", END)

# ---- 4. Compile Graph ----
app = workflow.compile()

# ---- 5. Run ----
query = "Divorce cases research mode"
state = AgentState(query=query, research_mode=True)

result = app.invoke(state)

print("Answer:", result.get("answer"))
print("Structured Report:", result.get("report"))


Answer: Research mode for divorce cases would involve accessing and analyzing legal databases (like Westlaw, LexisNexis), court records (often online), scholarly articles on divorce law and related social issues (e.g., family dynamics, child custody), and potentially consulting with legal professionals or academic experts. The specific focus depends on the research question.
Structured Report: None


In [1]:
app

NameError: name 'app' is not defined

In [16]:
# ---- 3. Build the Graph ----
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("direct_answer", direct_answer)
workflow.add_node("retriever", retriever)
workflow.add_node("summarizer", summarizer)
workflow.add_node("formatter", formatter)

# Conditional branch function (no changes needed here)
def decide_branch(state: AgentState):
    """
    Checks the 'research_mode' flag in the state to decide the next step.
    """
    if state.get("research_mode"):
        print("---RESEARCH MODE DETECTED, ROUTING TO RETRIEVER---")
        return "retriever"
    else:
        print("---DIRECT ANSWER MODE DETECTED, ROUTING TO DIRECT ANSWER---")
        return "direct_answer"

# **THE FIX:** Make the conditional branch the entry point of the graph.
workflow.set_conditional_entry_point(
    decide_branch,
    {
        "retriever": "retriever",
        "direct_answer": "direct_answer",
    }
)

# Define the rest of the graph edges
workflow.add_edge("retriever", "summarizer")
workflow.add_edge("summarizer", "formatter")
workflow.add_edge("formatter", END)
workflow.add_edge("direct_answer", END) # The direct path should also end the graph

# ---- 4. Compile Graph ----
app = workflow.compile()

# ---- 5. Run ----
# The input state is correct, with research_mode=True
query = "Can a Hindu marriage be dissolved on the ground of irretrievable breakdown?"
state = AgentState(query=query, research_mode=True)

# Invoke the app
result = app.invoke(state)

print("\n\n---FINAL OUTPUT---")
print("Answer:", result.get("answer"))
print("Structured Report:", result.get("report"))

---RESEARCH MODE DETECTED, ROUTING TO RETRIEVER---


---FINAL OUTPUT---
Answer: None
Structured Report: Okay, here's a breakdown of the provided information, formatted into the requested sections:

**Issue**

Can a Hindu marriage be dissolved on the ground of irretrievable breakdown? This is the central legal question addressed by the listed cases. The query suggests that the cases explore whether the "irretrievable breakdown of marriage" can be a valid ground for divorce under Hindu law.

**Relevant Case Law**

The following cases are potentially relevant to the issue:

*   **Shilpa Sailesh vs Varun Sreenivasan (Supreme Court, 2023)**
*   **Shri Rakesh Raman vs Smt. Kavita (Supreme Court, 2023)**
*   **Tapan Kumar Chakraborty vs Smt. Jyotsna Chakraborty (Calcutta High Court, 1996)**
*   **Disha Kushwaha vs Rituraj Singh (Madhya Pradesh High Court, 2019)**
*   **Bhavna Shamra vs Sri Sanjeev Sharma (Allahabad High Court, 2019)**
*   **Smiti W/O. Aakash Shah D/O. Atulbhai ... vs Aakash K

In [2]:
def fetch_kanoon_data(user_input="cases related to divorce by husband",api_token="14643c2a5b137ab178234409a2e8b8e219629d60",pagenum=0):
    url = "https://api.indiankanoon.org/search/"
    headers = {"Authorization": f"Token {api_token}", "Accept": "application/json"}
    params = {"formInput": user_input, "pagenum": pagenum}
    try:
        response = requests.post(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"API request failed: {e}")
    
import requests
print(fetch_kanoon_data(),"/n")

{'categories': [['Filter by AI Tags', [{'value': 'dowry', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20doctypes%3A%20judgments+tag:dowry'}, {'value': 'law-for-hindu-marriages', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20doctypes%3A%20judgments+tag:law-for-hindu-marriages'}, {'value': 'indian-penal-code', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20doctypes%3A%20judgments+tag:indian-penal-code'}, {'value': 'domestic-violence', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20doctypes%3A%20judgments+tag:domestic-violence'}, {'value': 'criminal-procedure-code', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20doctypes%3A%20judgments+tag:criminal-procedure-code'}, {'value': 'maintenance-of-wife-children-parents', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20doctypes%3A%20judgments+tag:maintenance-of-wife-children-parents'}, {'value': 'divorce', 'formInput': '%20%20related%20to%20divorce%20by%20husband%20do

In [ ]:
import requests

def fetch_kanoon_data(user_input, api_token, pagenum=0):
    url = "https://api.indiankanoon.org/search/"
    headers = {"Authorization": f"Token {api_token}", "Accept": "application/json"}
    params = {"formInput": user_input, "pagenum": pagenum}
    try:
        response = requests.post(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Extract relevant fields and generate case links
        results = []
        for doc in data.get('docs', []):
            case_info = {
                'tid': doc.get('tid'),
                'title': doc.get('title'),
                'docsource': doc.get('docsource'),
                'publishdate': doc.get('publishdate'),
                'author': doc.get('author'),
                # 'bench': doc.get('bench'),
                # 'catids': doc.get('catids'),
                'citation': doc.get('citation'),
                'numcites': doc.get('numcites'),
                'numcitedby': doc.get('numcitedby'),
                'link': f"https://indiankanoon.org/doc/{doc.get('tid')}/"
            }
            results.append(case_info)
        
        return {
            'cases': results,
            'found': data.get('found'),
            'encodedformInput': data.get('encodedformInput')
        }
    except requests.exceptions.RequestException as e:
        raise Exception(f"API request failed: {e}")
    
api_token = "14643c2a5b137ab178234409a2e8b8e219629d60"
result = fetch_kanoon_data("cases related to divorce by husband", api_token)
for case in result['cases']:
    print(f"Title: {case['title']}")
    print(f"Link: {case['link']}")
    print(f"TID: {case['tid']}")
    print(f"Court: {case['docsource']}")
    print(f"Date: {case['publishdate']}")
    print(f"Author: {case['author']}")
    print(f"Bench: {case['bench']}")
    print(f"Categories: {case['catids']}")
    print(f"Citation: {case['citation']}")
    print(f"Cites: {case['numcites']}, Cited By: {case['numcitedby']}")
    print("---")

Title: Smriti Madan Kansagra vs Perry Kansagra on 28 October, 2020
Link: https://indiankanoon.org/doc/181712544/
TID: 181712544
Court: Supreme Court of India
Date: 2020-10-28
Author: I Malhotra
Bench: [1719, 1736, 1219]
Categories: [52, 2047]
Citation: AIRONLINE 2020 SC 875
Cites: 51, Cited By: 33
---
Title: Pavakkal Noble John vs Kerala State on 3 September, 2010
Link: https://indiankanoon.org/doc/397599/
TID: 397599
Court: Kerala High Court
Date: 2010-09-03
Author: S Jagan
Bench: [1179]
Categories: [83]
Citation: None
Cites: 56, Cited By: 23
---
Title: Sandip Shankarlal Kedia vs Pooja Kedia on 29 April, 2013
Link: https://indiankanoon.org/doc/65326647/
TID: 65326647
Court: Bombay High Court
Date: 2013-04-29
Author: R Dalvi
Bench: [1896]
Categories: None
Citation: None
Cites: 86, Cited By: 4
---
Title: Allabuksh Karim Shaikh vs Smt. Noorjahan Allabuksh Shaikh And ... on 22 June, 1994
Link: https://indiankanoon.org/doc/1933807/
TID: 1933807
Court: Bombay High Court
Date: 1994-06-22
Aut

In [7]:
def fetch_case_details(tid=181712544, api_token="14643c2a5b137ab178234409a2e8b8e219629d60"):
    url = f"https://api.indiankanoon.org/doc/{tid}/"
    headers = {"Authorization": f"Token {api_token}", "Accept": "application/json"}
    try:
        response = requests.post(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"API request failed: {e}")
    
data=(fetch_case_details())
print(data['doc'])


<h2 class="doc_title">Smriti Madan Kansagra vs Perry Kansagra on 28 October, 2020</h2>

<h3 class="doc_citations">Equivalent citations: AIRONLINE 2020 SC 875</h3>

<h3 class="doc_author">Author: <a href="/search/?formInput=authorid:i-malhotra">Indu Malhotra</a></h3>

<h3 class="doc_bench">Bench: <a href="/search/?formInput=benchid:u-u-lalit">Uday Umesh Lalit</a>, <a href="/search/?formInput=benchid:i-malhotra">Indu Malhotra</a>, <a href="/search/?formInput=benchid:h-gupta">Hemant Gupta</a></h3>

<pre id="pre_1">                                                                           REPORTABLE


                                      IN THE SUPREME COURT OF INDIA
                                       CIVIL APPELLATE JURISDICTION

                                       CIVIL APPEAL NO. 3559 OF 2020
                                    (Arising out of SLP (C) No. 12910/ 2020
                                            (Diary No.8161 of 2020)


          Smriti Madan Kansagra            

In [ ]:
data['doc']